In [1]:
import torch
import json
import re

from collections import OrderedDict

from datasets import load_dataset
from tqdm import tqdm
from data.discovery_con import LABELS

from transformers import AutoModelForCausalLM, AutoTokenizer
from generate import DatasetGenerate

ImportError: cannot import name 'DiscoveryDatasetGenerate' from 'generate' (/home/nlp/apex/commonsense-discourse/generate.py)

In [143]:
model_path = '/home/nlp/apex/experiment/ctrl/'

In [11]:
dataset = load_dataset('data/discovery.py', 'discovery')

Reusing dataset discovery (/home/nlp/.cache/huggingface/datasets/discovery/discovery/1.0.0/f08ced5950fb93854c70d20fc70d1583766d7219cda67e65d197dfe9ec3775ca)


In [305]:
discovery_train_ds = load_dataset('data/discovery.py', 'discovery', split='train[:2%]') #7
discovery_valid_ds = dataset["validation"]
discovery_test_ds = dataset["test"]

Reusing dataset discovery (/home/nlp/.cache/huggingface/datasets/discovery/discovery/1.0.0/f08ced5950fb93854c70d20fc70d1583766d7219cda67e65d197dfe9ec3775ca)


In [306]:
len(discovery_train_ds)

31320

In [307]:
model = AutoModelForCausalLM.from_pretrained(model_path)

In [308]:
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                         max_length=64,
                                         padding='max_length',
                                         return_length=True,
                                         add_special_tokens=True)

In [309]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'cls_token': '[CLS]'})
tokenizer.add_special_tokens({'sep_token': '[SEP]'})

1

In [310]:
re.sub('[^A-Za-z0-9 ]+', '', 'dasdsa\xad dasas2321 }{[]`')

'dasdsa dasas2321 '

In [312]:
decoding_options_0 = {'max_length': 64,
                    'repetition_penalty': 1.2,
                    'temperature': 0}

decoding_options_1 = {'max_length': 64,
                      'num_beams':5, 
                      'no_repeat_ngram_size':2, 
                      'early_stopping':True}

decoding_options_2 = {'max_length': 64,
                    'do_sample':True, 
                    'max_length':64, 
                    'top_k':50, 
                    'top_p':0.95}

fallback_decoding = {'max_length': 64,
                     'num_beams':25, 
                     'no_repeat_ngram_size':2,
                     'num_return_sequences': 2,
                     'temperature': 0.7,
                     'early_stopping':True}

In [313]:
decoding_options = []
decoding_options.append(decoding_options_0)
decoding_options.append(decoding_options_1)
decoding_options.append(decoding_options_2)
decoding_options.append(fallback_decoding)

In [314]:
discovery_ds = DiscoveryDatasetGenerate(discovery_train_ds, LABELS, tokenizer, model, decoding_options)

In [315]:
synthetic_dataset = []

for i in tqdm(range(len(discovery_train_ds))):
    example = {}
    values = discovery_train_ds[i]
    example['context'] = values['sentence1']
    example['marker'] = LABELS[values['label']]
    generated_options = discovery_ds.generate_synthetic_options(i)
    example.update(generated_options)
    synthetic_dataset.append(example)

  0%|          | 4/31320 [00:17<38:52:04,  4.47s/it]

Detected an empty greedy output. Count:  1


  0%|          | 5/31320 [00:25<44:48:17,  5.15s/it]


KeyboardInterrupt: 

In [ ]:
synthetic_dataset[15]

In [ ]:
model.cuda();

In [159]:
decoding_options_0 = {'max_length': 64,
                    'repetition_penalty': 1.2}

decoding_options_1 = {'max_length': 64,
                      'num_beams':5, 
                      'no_repeat_ngram_size':2, 
                      'early_stopping':True}

decoding_options_2 = {'max_length': 64,
                    'do_sample':True, 
                    'max_length':64, 
                    'top_k':50, 
                    'top_p':0.85}

In [138]:
text = "curiously they continued to dig noting that there were pick marks on the walls of the pit where someone before them had dug out the pit."

In [139]:
def put_on_cuda(tok_output):
    for k, v in tok_output.items():
        tok_output[k] = v.cuda()
    return tok_output
cuda_token = put_on_cuda(tokenizer(text, return_tensors="pt"))

In [169]:
tokenizer.decode(model.generate(input_ids = cuda_token['input_ids'], **decoding_options_1).tolist()[0], skip_special_tokens=True)

"curiously they continued to dig noting that there were pick marks on the walls of the pit where someone before them had dug out the pit. `` The pit was filled in with earth and there was no sign of anyone having been in it before they filled it in. '' `'was told that the"

In [181]:
k = model.generate(input_ids = cuda_token['input_ids'], **fallback_decoding)

In [47]:
with open('data/synthetic_discovery_1500.json', 'w') as fout:
    json.dump(synthetic_dataset , fout)

In [8]:
discovery_train_ds.column_names[0]

'idx'